In [67]:
!pip install -U finance-datareader

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [68]:
import pandas as pd
import os
from tqdm import tqdm
import FinanceDataReader as fdr

# 파일별로 분류

In [69]:
import glob

재무상태표 = [] # _01_
손익계산서 = [] # _02_
포괄손익계산서 = [] # _03_
현금흐름표 = [] # _04_
자본변동표 = [] #_05_


for name in glob.glob('/content/drive/MyDrive/22캡스톤디자인/재무일괄/업데이트/*.txt'):
  if '_01_' in name:
    재무상태표.append(name)
  elif '_02_' in name:
    손익계산서.append(name)
  elif '_03_' in name:
    포괄손익계산서.append(name)
  elif '_04_' in name:
    현금흐름표.append(name)
  elif '_05_' in name:
    자본변동표.append(name)

In [70]:
sample = pd.DataFrame(columns=['종목코드','사업연도', '보고서종류', '회사명', '업종', '항목코드','항목명', '당기'])

# 데이터 추출

## 현금흐름표

In [71]:
현금흐름샘플 = pd.DataFrame(columns=['종목코드','사업연도', '보고서종류', '회사명', '업종', '항목코드','항목명', '당기'])
for path in tqdm(현금흐름표):
  df = pd.read_csv(path, delimiter = '\t', encoding='cp949')

  당기 = ''
  if df['보고서종류'][0] == '1분기보고서':
    당기 = '당기 1분기' 
  elif df['보고서종류'][0] == '반기보고서':
    당기 = '당기 반기' 
  elif df['보고서종류'][0] == '3분기보고서':
    당기 = '당기 3분기' 
  elif df['보고서종류'][0] == '사업보고서':
    당기 = '당기' 

  df['사업연도'] = path[57:61] # 사업연도 컬럼 설정 - 파일명에서 가져온다
  df['항목명'] = df['항목명'].str.strip() # 항목명 컬럼 문자열 전처리
  df['항목코드'] = df['항목코드'].str.strip() # 항목코드 컬럼 문자열 전처리

  df = df[['종목코드','사업연도', '보고서종류', '회사명', '업종', '항목코드','항목명', 당기]]

  전체종목 = len(df['종목코드'].unique())

  # 영업활동현금흐름(ifrs_CashFlowsFromUsedInOperatingActivities)
  영업활동현금흐름 = df[(df['항목코드'] == 'ifrs-full_CashFlowsFromUsedInOperatingActivities') | (df['항목코드'] == 'ifrs_CashFlowsFromUsedInOperatingActivities') | (df['항목명'].str.contains('영업활동'))]
  영업활동현금흐름.rename(columns={당기:"당기"}, inplace=True)
  영업활동현금흐름.drop_duplicates(['종목코드','사업연도','보고서종류','항목명'])
  현금흐름샘플 = 현금흐름샘플.append(영업활동현금흐름, sort=False, ignore_index=True)
  

  # 투자활동현금흐름(ifrs_CashFlowsFromUsedInInvestingActivities)
  투자활동현금흐름 = df[(df['항목코드'] == 'ifrs_CashFlowsFromUsedInInvestingActivities') | (df['항목코드'] == 'ifrs-full_CashFlowsFromUsedInInvestingActivities') | (df['항목명'].str.contains('투자활동'))]
  투자활동현금흐름.rename(columns={당기:"당기"}, inplace=True)
  투자활동현금흐름.drop_duplicates(['종목코드','사업연도','보고서종류','항목명'])
  현금흐름샘플 = 현금흐름샘플.append(투자활동현금흐름, sort=False, ignore_index=True)

  # 재무활동현금흐름(ifrs_CashFlowsFromUsedInFinancingActivities)
  재무활동현금흐름 = df[(df['항목코드'] == 'ifrs_CashFlowsFromUsedInFinancingActivities') | (df['항목코드'] == 'ifrs-full_CashFlowsFromUsedInFinancingActivities') |(df['항목명'].str.contains('재무활동'))]
  재무활동현금흐름.rename(columns={당기:"당기"}, inplace=True)
  재무활동현금흐름.drop_duplicates(['종목코드','사업연도','보고서종류','항목명'])
  현금흐름샘플 = 현금흐름샘플.append(재무활동현금흐름, sort=False, ignore_index=True)


  0%|          | 0/2 [00:00<?, ?it/s]/usr/local/lib/python3.7/dist-packages/pandas/core/frame.py:5047: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,
100%|██████████| 2/2 [00:02<00:00,  1.13s/it]


In [72]:
현금흐름샘플

,종목코드,사업연도,보고서종류,회사명,업종,항목코드,항목명,당기
0,[060310],2021,사업보고서,3S,292,ifrs-full_CashFlowsFromUsedInOperatingActivities,영업활동으로 인한 현금흐름,"871,088,551"
1,[060310],2021,사업보고서,3S,292,dart_AdjustmentsForAssetsLiabilitiesOfOperatin...,영업활동으로 인한 자산부채의 변동,"574,640,381"
2,[095570],2021,사업보고서,AJ네트웍스,763,ifrs-full_CashFlowsFromUsedInOperatingActivities,영업활동 현금흐름,"-11,318,072,318"
3,[006840],2021,사업보고서,AK홀딩스,649,ifrs-full_CashFlowsFromUsedInOperatingActivities,영업활동현금흐름,"-9,766,237,791"
4,[054620],2021,사업보고서,APS홀딩스,649,ifrs-full_CashFlowsFromUsedInOperatingActivities,영업활동현금흐름,"-1,568,988,010"
...,...,...,...,...,...,...,...,...
14863,[003280],2022,1분기보고서,흥아해운,501,ifrs-full_CashFlowsFromUsedInFinancingActivities,재무활동현금흐름,"-3,544,038,131"
14864,[037440],2022,1분기보고서,희림,721,ifrs-full_CashFlowsFromUsedInFinancingActivities,재무활동현금흐름,"-2,162,502,506"
14865,[238490],2022,1분기보고서,힘스,292,ifrs-full_CashFlowsFromUsedInFinancingActivities,재무활동현금흐름,"-254,631,420"
14866,[238490],2022,1분기보고서,힘스,292,entity00556712_udf_CF_20201022103737527_CashFl...,재무활동으로 인한 현금 유입액,"2,001,000,000"


## 손익계산서

In [73]:
손익계산샘플 = pd.DataFrame(columns=['종목코드','사업연도', '보고서종류', '회사명', '업종', '항목코드','항목명', '당기'])
for path in tqdm(손익계산서):
  df = pd.read_csv(path, delimiter = '\t', encoding='cp949')

  당기 = ''
  if df['보고서종류'][0] == '1분기보고서':
    당기 = '당기 1분기 누적' 
  elif df['보고서종류'][0] == '반기보고서':
    당기 = '당기 반기 누적' 
  elif df['보고서종류'][0] == '3분기보고서':
    당기 = '당기 3분기 누적' 
  elif df['보고서종류'][0] == '사업보고서':
    당기 = '당기' 

  df['사업연도'] = path[57:61] # 사업연도 컬럼 d설정 - 파일명에서 가져온다
  df['항목명'] = df['항목명'].str.strip() # 항목명 컬럼 문자열 전처리
  df['항목코드'] = df['항목코드'].str.strip() # 항목코드 컬럼 문자열 전처리

  df = df[['종목코드','사업연도', '보고서종류', '회사명', '업종', '항목코드','항목명', 당기]]

  전체종목 = len(df['종목코드'].unique())

  # (수익)매출액 (ifrs-full_Revenue)
  매출액 = df[(df['항목코드'] == 'ifrs_Revenue') | (df['항목코드'] == 'ifrs-full_Revenue') | (df['항목명'].isin(['매출액','(수익)매출액','매출액(수익)','수익(매출액)']))]
  매출액.rename(columns={당기:"당기"}, inplace=True)
  매출액.drop_duplicates(['종목코드','사업연도','보고서종류','항목명'])
  손익계산샘플 = 손익계산샘플.append(매출액, sort=False, ignore_index=True)


  # 매출총이익 (ifrs-full_GrossProfit))
  매출총이익 = df[(df['항목코드'] == 'ifrs_GrossProfit') | (df['항목코드'] == 'ifrs-full_GrossProfit')| (df['항목명'].isin(['매출총이익','(수익)매출총이익','매출총이익(손실)']))]
  매출총이익.rename(columns={당기:"당기"}, inplace=True)
  매출총이익.drop_duplicates(['종목코드','사업연도','보고서종류','항목명'])
  손익계산샘플 = 손익계산샘플.append(매출총이익, sort=False, ignore_index=True)

  # 영업이익(손실) (dart_OperatingIncomeLoss)
  영업이익 = df[(df['항목코드'] == 'dart_OperatingIncomeLoss')|(df['항목명'].isin(['영업이익','영업이익(손실)']))]
  영업이익.rename(columns={당기:"당기"}, inplace=True)
  영업이익.drop_duplicates(['종목코드','사업연도','보고서종류','항목명'])
  손익계산샘플 = 손익계산샘플.append(영업이익, sort=False, ignore_index=True)
  
  # 당기순이익(손실) (ifrs_ProfitLoss)
  당기순이익 = df[(df['항목코드'] == 'ifrs-full_ProfitLoss') | (df['항목코드'] == 'ifrs_ProfitLoss') |  (df['항목명'].isin(['당기순이익','당기순이익(손실)']))]
  당기순이익.rename(columns={당기:"당기"}, inplace=True)
  당기순이익.drop_duplicates(['종목코드','사업연도','보고서종류','항목명'])
  손익계산샘플 = 손익계산샘플.append(당기순이익, sort=False, ignore_index=True)

  # 비지배지분(ifrs_ProfitLossAttributableToNoncontrollingInterests)
  비지배지분 = df[(df['항목코드'] == 'ifrs-full_ProfitLossAttributableToNoncontrollingInterests') | (df['항목코드'] == 'ifrs_ProfitLossAttributableToNoncontrollingInterests') | (df['항목명'].str.contains('비지배'))]
  비지배지분.rename(columns={당기:"당기"}, inplace=True)
  비지배지분.drop_duplicates(['종목코드','사업연도','보고서종류','항목명'])
  손익계산샘플 = 손익계산샘플.append(비지배지분, sort=False, ignore_index=True)

  # 감가상각비(dart_DepreciationExpense)
  감가상각비 = df[(df['항목코드'] == 'dart_DepreciationExpense')|(df['항목명'].isin(['감가상각비']))]
  감가상각비.rename(columns={당기:"당기"}, inplace=True)
  손익계산샘플 = 손익계산샘플.append(감가상각비, sort=False, ignore_index=True)

  # 법인세비용차감전순이익(손실)(ifrs-full_ProfitLossBeforeTax)
  법인세차감전순이익 = df[(df['항목코드'] == 'ifrs-full_ProfitLossBeforeTax') | (df['항목코드'] == 'ifrs_ProfitLossBeforeTax') | df['항목명'].isin(['법인세차감전순이익','법인세차감전순이익(손실)'])]
  법인세차감전순이익.rename(columns={당기:"당기"}, inplace=True)
  법인세차감전순이익.drop_duplicates(['종목코드','사업연도','보고서종류','항목명'])
  손익계산샘플 = 손익계산샘플.append(법인세차감전순이익, sort=False, ignore_index=True)
  

  0%|          | 0/2 [00:00<?, ?it/s]/usr/local/lib/python3.7/dist-packages/pandas/core/frame.py:5047: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,
100%|██████████| 2/2 [00:00<00:00, 11.55it/s]


In [74]:
손익계산샘플

,종목코드,사업연도,보고서종류,회사명,업종,항목코드,항목명,당기
0,[001040],2021,사업보고서,CJ,649,ifrs-full_Revenue,매출액,"34,484,026,554,000"
1,[000120],2021,사업보고서,CJ대한통운,493,ifrs-full_Revenue,매출액,"11,343,651,787,492"
2,[000990],2021,사업보고서,DB하이텍,261,ifrs-full_Revenue,매출액,"1,214,682,040,178"
3,[000210],2021,사업보고서,DL,649,ifrs-full_Revenue,매출액,"2,385,420,753,496"
4,[375500],2021,사업보고서,DL이앤씨,412,ifrs-full_Revenue,매출액,"7,631,650,898,582"
...,...,...,...,...,...,...,...,...
1931,[008770],2022,1분기보고서,호텔신라,478,ifrs-full_ProfitLossBeforeTax,VII. 법인세비용차감전순손익,"-7,795,875,667"
1932,[004800],2022,1분기보고서,효성,649,ifrs-full_ProfitLossBeforeTax,법인세비용차감전 계속영업이익(손실),"72,969,011,271"
1933,[298050],2022,1분기보고서,효성첨단소재,205,ifrs-full_ProfitLossBeforeTax,법인세비용차감전순이익(손실),"93,763,702,404"
1934,[298020],2022,1분기보고서,효성티앤씨,205,ifrs-full_ProfitLossBeforeTax,법인세비용차감전순이익,"159,571,477,299"


## 포괄손익계산서

In [75]:
포괄손익계산샘플 = pd.DataFrame(columns=['종목코드','사업연도', '보고서종류', '회사명', '업종', '항목코드','항목명', '당기'])
for path in tqdm(포괄손익계산서):
  df = pd.read_csv(path, delimiter = '\t', encoding='cp949')

  당기 = ''
  if df['보고서종류'][0] == '1분기보고서':
    당기 = '당기 1분기 누적' 
  elif df['보고서종류'][0] == '반기보고서':
    당기 = '당기 반기 누적' 
  elif df['보고서종류'][0] == '3분기보고서':
    당기 = '당기 3분기 누적' 
  elif df['보고서종류'][0] == '사업보고서':
    당기 = '당기' 

  df['사업연도'] = path[57:61] # 사업연도 컬럼 d설정 - 파일명에서 가져온다
  df['항목명'] = df['항목명'].str.strip() # 항목명 컬럼 문자열 전처리
  df['항목코드'] = df['항목코드'].str.strip() # 항목코드 컬럼 문자열 전처리

  df = df[['종목코드','사업연도', '보고서종류', '회사명', '업종', '항목코드','항목명', 당기]]

  전체종목 = len(df['종목코드'].unique())

  # (수익)매출액 (ifrs-full_Revenue)
  매출액 = df[(df['항목코드'] == 'ifrs_Revenue') | (df['항목코드'] == 'ifrs-full_Revenue') | (df['항목명'].isin(['매출액','(수익)매출액','매출액(수익)','수익(매출액)']))]
  매출액.rename(columns={당기:"당기"}, inplace=True)
  매출액.drop_duplicates(['종목코드','사업연도','보고서종류','항목명'])
  포괄손익계산샘플 = 포괄손익계산샘플.append(매출액, sort=False, ignore_index=True)


  # 매출총이익 (ifrs-full_GrossProfit))
  매출총이익 = df[(df['항목코드'] == 'ifrs_GrossProfit') | (df['항목코드'] == 'ifrs-full_GrossProfit')| (df['항목명'].isin(['매출총이익','(수익)매출총이익','매출총이익(손실)']))]
  매출총이익.rename(columns={당기:"당기"}, inplace=True)
  매출총이익.drop_duplicates(['종목코드','사업연도','보고서종류','항목명'])
  포괄손익계산샘플 = 포괄손익계산샘플.append(매출총이익, sort=False, ignore_index=True)

  # 영업이익(손실) (dart_OperatingIncomeLoss)
  영업이익 = df[(df['항목코드'] == 'dart_OperatingIncomeLoss')|(df['항목명'].isin(['영업이익','영업이익(손실)']))]
  영업이익.rename(columns={당기:"당기"}, inplace=True)
  영업이익.drop_duplicates(['종목코드','사업연도','보고서종류','항목명'])
  포괄손익계산샘플 = 포괄손익계산샘플.append(영업이익, sort=False, ignore_index=True)
  
  # 당기순이익(손실) (ifrs_ProfitLoss)
  당기순이익 = df[(df['항목코드'] == 'ifrs-full_ProfitLoss') | (df['항목코드'] == 'ifrs_ProfitLoss') |  (df['항목명'].isin(['당기순이익','당기순이익(손실)']))]
  당기순이익.rename(columns={당기:"당기"}, inplace=True)
  당기순이익.drop_duplicates(['종목코드','사업연도','보고서종류','항목명'])
  포괄손익계산샘플 = 포괄손익계산샘플.append(당기순이익, sort=False, ignore_index=True)

  # 비지배지분(ifrs_ProfitLossAttributableToNoncontrollingInterests)
  비지배지분 = df[(df['항목코드'] == 'ifrs-full_ProfitLossAttributableToNoncontrollingInterests') | (df['항목코드'] == 'ifrs_ProfitLossAttributableToNoncontrollingInterests') 
    | (df['항목명'].str.contains('비지배')| (df['항목명'].isin(['비지배지분에 귀속되는 당기순이익', '비지배지분순이익'])))]
  비지배지분.rename(columns={당기:"당기"}, inplace=True)
  비지배지분.drop_duplicates(['종목코드','사업연도','보고서종류','항목명'])
  포괄손익계산샘플 = 포괄손익계산샘플.append(비지배지분, sort=False, ignore_index=True)

  # 감가상각비(dart_DepreciationExpense)
  감가상각비 = df[(df['항목코드'] == 'dart_DepreciationExpense')|(df['항목명'].isin(['감가상각비']))]
  감가상각비.rename(columns={당기:"당기"}, inplace=True)
  포괄손익계산샘플 = 포괄손익계산샘플.append(감가상각비, sort=False, ignore_index=True)

  # 법인세비용차감전순이익(손실)(ifrs-full_ProfitLossBeforeTax)
  법인세차감전순이익 = df[(df['항목코드'] == 'ifrs-full_ProfitLossBeforeTax') | (df['항목코드'] == 'ifrs_ProfitLossBeforeTax') | df['항목명'].isin(['법인세차감전순이익','법인세차감전순이익(손실)'])]
  법인세차감전순이익.rename(columns={당기:"당기"}, inplace=True)
  법인세차감전순이익.drop_duplicates(['종목코드','사업연도','보고서종류','항목명'])
  포괄손익계산샘플 = 포괄손익계산샘플.append(법인세차감전순이익, sort=False, ignore_index=True)

  0%|          | 0/2 [00:00<?, ?it/s]/usr/local/lib/python3.7/dist-packages/pandas/core/frame.py:5047: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,
100%|██████████| 2/2 [00:01<00:00,  1.36it/s]


In [76]:
mask = (포괄손익계산샘플['항목명'].str.contains('포괄|계속|중단|해외'))|(포괄손익계산샘플['항목코드'].str.contains('Operations|ComprehensiveIncome'))
포괄손익계산샘플 = 포괄손익계산샘플[~mask]

포괄손익계산샘플['항목명'] = 포괄손익계산샘플.apply(lambda x: '매출액' if x['항목코드'] == 'ifrs_Revenue' else x['항목명'], axis=1)
포괄손익계산샘플['항목명'] = 포괄손익계산샘플.apply(lambda x: '매출액' if x['항목코드'] == 'ifrs-full_Revenue' else x['항목명'], axis=1)
포괄손익계산샘플['항목명'] = 포괄손익계산샘플.apply(lambda x: '매출총이익' if x['항목코드'] == 'ifrs_GrossProfit' else x['항목명'], axis=1)
포괄손익계산샘플['항목명'] = 포괄손익계산샘플.apply(lambda x: '매출총이익' if x['항목코드'] == 'ifrs-full_GrossProfit' else x['항목명'], axis=1)
포괄손익계산샘플['항목명'] = 포괄손익계산샘플.apply(lambda x: '영업이익' if x['항목코드'] == 'dart_OperatingIncomeLoss' else x['항목명'], axis=1)
포괄손익계산샘플['항목명'] = 포괄손익계산샘플.apply(lambda x: '당기순이익' if x['항목코드'] == 'ifrs_ProfitLoss' else x['항목명'], axis=1)
포괄손익계산샘플['항목명'] = 포괄손익계산샘플.apply(lambda x: '당기순이익' if x['항목코드'] == 'ifrs-full_ProfitLoss' else x['항목명'], axis=1)

포괄손익계산샘플['항목명'] = 포괄손익계산샘플.apply(lambda x: '당기순이익(비지배)' if 'ProfitLossAttributableToNoncontrollingInterests' in x['항목코드'] else x['항목명'], axis=1)

포괄손익계산샘플['항목명'] = 포괄손익계산샘플.apply(lambda x: '감가상각비' if x['항목코드'] == 'dart_DepreciationExpense' else x['항목명'], axis=1)
포괄손익계산샘플['항목명'] = 포괄손익계산샘플.apply(lambda x: '법인세비용차감전순이익' if x['항목코드'] == 'ifrs_ProfitLossBeforeTax' else x['항목명'], axis=1)
포괄손익계산샘플['항목명'] = 포괄손익계산샘플.apply(lambda x: '법인세비용차감전순이익' if x['항목코드'] == 'ifrs-full_ProfitLossBeforeTax' else x['항목명'], axis=1)
포괄손익계산샘플['항목명'] = 포괄손익계산샘플.apply(lambda x: '매출액' if x['항목명'] == '수익(매출액)' else x['항목명'], axis=1)
포괄손익계산샘플['항목명'] = 포괄손익계산샘플.apply(lambda x: '매출총이익' if x['항목명'] == '매출총이익(손실)' else x['항목명'], axis=1)
포괄손익계산샘플['항목명'] = 포괄손익계산샘플.apply(lambda x: '영업이익' if x['항목명'] == '영업이익(손실)' else x['항목명'], axis=1)
포괄손익계산샘플['항목명'] = 포괄손익계산샘플.apply(lambda x: '당기순이익' if x['항목명'] == '당기순이익(손실)' else x['항목명'], axis=1)
포괄손익계산샘플['항목명'] = 포괄손익계산샘플.apply(lambda x: '법인세차감전순이익' if x['항목명'] == '법인세차감전순이익(손실)' else x['항목명'], axis=1)
포괄손익계산샘플['항목명'] = 포괄손익계산샘플.apply(lambda x: '법인세차감전순이익' if x['항목명'] == '법인세비용차감전순이익' else x['항목명'], axis=1)

포괄손익계산샘플

,종목코드,사업연도,보고서종류,회사명,업종,항목코드,항목명,당기
0,[060310],2021,사업보고서,3S,292,ifrs-full_Revenue,매출액,"23,477,908,383"
1,[095570],2021,사업보고서,AJ네트웍스,763,ifrs-full_Revenue,매출액,"1,026,749,457,003"
2,[006840],2021,사업보고서,AK홀딩스,649,ifrs-full_Revenue,매출액,"3,167,699,652,090"
3,[054620],2021,사업보고서,APS홀딩스,649,ifrs-full_Revenue,매출액,"39,415,521,257"
4,[265520],2021,사업보고서,AP시스템,292,ifrs-full_Revenue,매출액,"528,740,970,343"
...,...,...,...,...,...,...,...,...
22625,[010240],2022,1분기보고서,흥국,292,ifrs-full_ProfitLossBeforeTax,법인세차감전순이익,"3,353,383,900"
22626,[189980],2022,1분기보고서,흥국에프엔비,112,ifrs-full_ProfitLossBeforeTax,법인세차감전순이익,"1,236,152,104"
22627,[003280],2022,1분기보고서,흥아해운,501,ifrs-full_ProfitLossBeforeTax,법인세차감전순이익,"-371,037,287"
22628,[037440],2022,1분기보고서,희림,721,ifrs-full_ProfitLossBeforeTax,법인세차감전순이익,"1,745,202,772"


In [77]:
포괄손익계산샘플

,종목코드,사업연도,보고서종류,회사명,업종,항목코드,항목명,당기
0,[060310],2021,사업보고서,3S,292,ifrs-full_Revenue,매출액,"23,477,908,383"
1,[095570],2021,사업보고서,AJ네트웍스,763,ifrs-full_Revenue,매출액,"1,026,749,457,003"
2,[006840],2021,사업보고서,AK홀딩스,649,ifrs-full_Revenue,매출액,"3,167,699,652,090"
3,[054620],2021,사업보고서,APS홀딩스,649,ifrs-full_Revenue,매출액,"39,415,521,257"
4,[265520],2021,사업보고서,AP시스템,292,ifrs-full_Revenue,매출액,"528,740,970,343"
...,...,...,...,...,...,...,...,...
22625,[010240],2022,1분기보고서,흥국,292,ifrs-full_ProfitLossBeforeTax,법인세차감전순이익,"3,353,383,900"
22626,[189980],2022,1분기보고서,흥국에프엔비,112,ifrs-full_ProfitLossBeforeTax,법인세차감전순이익,"1,236,152,104"
22627,[003280],2022,1분기보고서,흥아해운,501,ifrs-full_ProfitLossBeforeTax,법인세차감전순이익,"-371,037,287"
22628,[037440],2022,1분기보고서,희림,721,ifrs-full_ProfitLossBeforeTax,법인세차감전순이익,"1,745,202,772"


## 재무상태표

In [78]:
재무상태샘플 = pd.DataFrame(columns=['종목코드','사업연도', '보고서종류', '회사명', '업종', '항목코드','항목명', '당기'])

for path in tqdm(재무상태표):
  df = pd.read_csv(path, delimiter = '\t', encoding='cp949')

  당기 = ''
  if df['보고서종류'][0] == '1분기보고서':
    당기 = '당기 1분기말' 
  elif df['보고서종류'][0] == '반기보고서':
    당기 = '당기 반기말' 
  elif df['보고서종류'][0] == '3분기보고서':
    당기 = '당기 3분기말' 
  elif df['보고서종류'][0] == '사업보고서':
    당기 = '당기' 

  df['사업연도'] = path[57:61] # 사업연도 컬럼 설정 - 파일명에서 가져온다
  df['항목명'] = df['항목명'].str.strip() # 항목명 컬럼 문자열 전처리
  df['항목코드'] = df['항목코드'].str.strip() # 항목코드 컬럼 문자열 전처리

  df = df[['종목코드','사업연도', '보고서종류', '회사명', '업종', '항목코드','항목명', 당기]]

  전체종목 = len(df['종목코드'].unique())

  # 자산총계(ifrs_Assets)
  자산총계 = df[(df['항목코드'] == 'ifrs_Assets') | (df['항목코드'] == 'ifrs-full_Assets') | (df['항목명'].isin(['자산 총계', '자산총계']))]
  자산총계.rename(columns={당기:"당기"}, inplace=True)
  자산총계.drop_duplicates(['종목코드','사업연도','보고서종류','항목명'])
  재무상태샘플 = 재무상태샘플.append(자산총계, sort=False, ignore_index=True)

  # 자본총계(ifrs_Equity)
  자본총계 = df[(df['항목코드'] == 'ifrs_Equity') | (df['항목코드'] == 'ifrs-full_Equity') | (df['항목명'].isin(['자본총계','자본 총계']))]
  자본총계.rename(columns={당기:"당기"}, inplace=True)
  자본총계.drop_duplicates(['종목코드','사업연도','보고서종류','항목명'])
  재무상태샘플 = 재무상태샘플.append(자본총계, sort=False, ignore_index=True)

  # 비지배지분(ifrs_NoncontrollingInterests)
  비지배지분 = df[(df['항목코드'] == 'ifrs-full_NoncontrollingInterests')|(df['항목코드'] == 'ifrs_NoncontrollingInterests')|(df['항목명'].str.contains('비지배'))]
  비지배지분.rename(columns={당기:"당기"}, inplace=True)
  비지배지분.drop_duplicates(['종목코드','사업연도','보고서종류','항목명'])
  재무상태샘플 = 재무상태샘플.append(비지배지분, sort=False, ignore_index=True)

  # 부채총계(ifrs_Liabilities)
  부채총계 = df[(df['항목코드'] == 'ifrs_Liabilities') | (df['항목코드'] == 'ifrs-full_Liabilities') | (df['항목명'].isin(['부채총계','부채 총계']))]
  부채총계.rename(columns={당기:"당기"}, inplace=True)
  부채총계.drop_duplicates(['종목코드','사업연도','보고서종류','항목명'])
  재무상태샘플 = 재무상태샘플.append(부채총계, sort=False, ignore_index=True)

  # 유동자산(ifrs_CurrentAssets)
  유동자산 = df[(df['항목코드'] == 'ifrs_CurrentAssets') | (df['항목코드'] == 'ifrs-full_CurrentAssets') | (df['항목명'].isin(['유동자산', '유동 자산']))]
  유동자산.rename(columns={당기:"당기"}, inplace=True)
  유동자산.drop_duplicates(['종목코드','사업연도','보고서종류','항목명'])
  재무상태샘플 = 재무상태샘플.append(유동자산, sort=False, ignore_index=True)

  # 현금 및 현금성자산(ifrs_CashAndCashEquivalents)
  현금성자산 = df[(df['항목코드'] == 'ifrs_CashAndCashEquivalents') | (df['항목코드'] == 'ifrs-full_CashAndCashEquivalents')|df['항목명'].isin(['현금', '현금성자산', '현금 및 현금성자산', '현금및현금성자산'])|df['항목명'].str.contains('현금및현금성자산|현금 및 현금성자산')]
  현금성자산.rename(columns={당기:"당기"}, inplace=True)
  현금성자산.drop_duplicates(['종목코드','사업연도','보고서종류','항목명'])
  재무상태샘플 = 재무상태샘플.append(현금성자산, sort=False, ignore_index=True)


  0%|          | 0/2 [00:00<?, ?it/s]/usr/local/lib/python3.7/dist-packages/pandas/core/frame.py:5047: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,
100%|██████████| 2/2 [00:03<00:00,  1.73s/it]


In [79]:
재무상태샘플

,종목코드,사업연도,보고서종류,회사명,업종,항목코드,항목명,당기
0,[060310],2021,사업보고서,3S,292,ifrs-full_Assets,자산총계,"54,614,752,349"
1,[095570],2021,사업보고서,AJ네트웍스,763,ifrs-full_Assets,자산총계,"1,355,044,449,955"
2,[006840],2021,사업보고서,AK홀딩스,649,ifrs-full_Assets,자산총계,"4,548,736,656,905"
3,[054620],2021,사업보고서,APS홀딩스,649,ifrs-full_Assets,자산총계,"337,092,577,346"
4,[265520],2021,사업보고서,AP시스템,292,ifrs-full_Assets,자산총계,"428,720,088,928"
...,...,...,...,...,...,...,...,...
21567,[010240],2022,1분기보고서,흥국,292,ifrs-full_CashAndCashEquivalents,현금및현금성자산,"16,028,579,778"
21568,[189980],2022,1분기보고서,흥국에프엔비,112,ifrs-full_CashAndCashEquivalents,현금및현금성자산,"17,648,654,115"
21569,[003280],2022,1분기보고서,흥아해운,501,ifrs-full_CashAndCashEquivalents,현금및현금성자산,"48,758,422,117"
21570,[037440],2022,1분기보고서,희림,721,ifrs-full_CashAndCashEquivalents,현금및현금성자산,"26,045,207,278"


# 데이터 전처리

## 현금흐름표

In [80]:
현금흐름샘플['항목명'] = 현금흐름샘플.apply(lambda x: '영업활동현금흐름' if x['항목코드'] == 'ifrs_CashFlowsFromUsedInOperatingActivities' else x['항목명'], axis=1)
현금흐름샘플['항목명'] = 현금흐름샘플.apply(lambda x: '영업활동현금흐름' if x['항목코드'] == 'ifrs-full_CashFlowsFromUsedInOperatingActivities' else x['항목명'], axis=1)
현금흐름샘플['항목명'] = 현금흐름샘플.apply(lambda x: '투자활동현금흐름' if x['항목코드'] == 'ifrs_CashFlowsFromUsedInInvestingActivities' else x['항목명'], axis=1)
현금흐름샘플['항목명'] = 현금흐름샘플.apply(lambda x: '투자활동현금흐름' if x['항목코드'] == 'ifrs-full_CashFlowsFromUsedInInvestingActivities' else x['항목명'], axis=1)
현금흐름샘플['항목명'] = 현금흐름샘플.apply(lambda x: '재무활동현금흐름' if x['항목코드'] == 'ifrs_CashFlowsFromUsedInFinancingActivities' else x['항목명'], axis=1)
현금흐름샘플['항목명'] = 현금흐름샘플.apply(lambda x: '재무활동현금흐름' if x['항목코드'] == 'ifrs-full_CashFlowsFromUsedInFinancingActivities' else x['항목명'], axis=1)

현금흐름샘플

,종목코드,사업연도,보고서종류,회사명,업종,항목코드,항목명,당기
0,[060310],2021,사업보고서,3S,292,ifrs-full_CashFlowsFromUsedInOperatingActivities,영업활동현금흐름,"871,088,551"
1,[060310],2021,사업보고서,3S,292,dart_AdjustmentsForAssetsLiabilitiesOfOperatin...,영업활동으로 인한 자산부채의 변동,"574,640,381"
2,[095570],2021,사업보고서,AJ네트웍스,763,ifrs-full_CashFlowsFromUsedInOperatingActivities,영업활동현금흐름,"-11,318,072,318"
3,[006840],2021,사업보고서,AK홀딩스,649,ifrs-full_CashFlowsFromUsedInOperatingActivities,영업활동현금흐름,"-9,766,237,791"
4,[054620],2021,사업보고서,APS홀딩스,649,ifrs-full_CashFlowsFromUsedInOperatingActivities,영업활동현금흐름,"-1,568,988,010"
...,...,...,...,...,...,...,...,...
14863,[003280],2022,1분기보고서,흥아해운,501,ifrs-full_CashFlowsFromUsedInFinancingActivities,재무활동현금흐름,"-3,544,038,131"
14864,[037440],2022,1분기보고서,희림,721,ifrs-full_CashFlowsFromUsedInFinancingActivities,재무활동현금흐름,"-2,162,502,506"
14865,[238490],2022,1분기보고서,힘스,292,ifrs-full_CashFlowsFromUsedInFinancingActivities,재무활동현금흐름,"-254,631,420"
14866,[238490],2022,1분기보고서,힘스,292,entity00556712_udf_CF_20201022103737527_CashFl...,재무활동으로 인한 현금 유입액,"2,001,000,000"


In [81]:
현금흐름샘플['종목코드'] = 현금흐름샘플['종목코드'].apply(lambda x:x[1:-1])
mask = 현금흐름샘플['항목명'].str.contains('자산|부채|기타|유출|유입|중단|연결|지급|수취|손실|제거|전|관련|조정|계속')
현금흐름샘플 = 현금흐름샘플[~mask]

In [82]:
현금흐름샘플['항목명'] = 현금흐름샘플.apply(lambda x: '영업활동현금흐름' if 'CashFlowsFromUsedInOperatingActivities' in x['항목코드'] else x['항목명'], axis=1)
현금흐름샘플['항목명'] = 현금흐름샘플.apply(lambda x: '투자활동현금흐름' if 'CashFlowsFromUsedInInvestingActivities' in x['항목코드'] else x['항목명'], axis=1)
현금흐름샘플['항목명'] = 현금흐름샘플.apply(lambda x: '재무활동현금흐름' if 'CashFlowsFromUsedInFinancingActivities' in x['항목코드'] else x['항목명'], axis=1)

현금흐름샘플['항목코드'] = 현금흐름샘플.apply(lambda x: 'ifrs_CashFlowsFromUsedInOperatingActivities' if 'CashFlowsFromUsedInOperatingActivities' in x['항목코드'] else x['항목코드'], axis=1)
현금흐름샘플['항목코드'] = 현금흐름샘플.apply(lambda x: 'ifrs_CashFlowsFromUsedInInvestingActivities' if 'CashFlowsFromUsedInInvestingActivities' in x['항목코드'] else x['항목코드'], axis=1)
현금흐름샘플['항목코드'] = 현금흐름샘플.apply(lambda x: 'ifrs_CashFlowsFromUsedInFinancingActivities' if 'CashFlowsFromUsedInFinancingActivities' in x['항목코드'] else x['항목코드'], axis=1)

현금흐름샘플

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documen

,종목코드,사업연도,보고서종류,회사명,업종,항목코드,항목명,당기
0,060310,2021,사업보고서,3S,292,ifrs_CashFlowsFromUsedInOperatingActivities,영업활동현금흐름,"871,088,551"
2,095570,2021,사업보고서,AJ네트웍스,763,ifrs_CashFlowsFromUsedInOperatingActivities,영업활동현금흐름,"-11,318,072,318"
3,006840,2021,사업보고서,AK홀딩스,649,ifrs_CashFlowsFromUsedInOperatingActivities,영업활동현금흐름,"-9,766,237,791"
4,054620,2021,사업보고서,APS홀딩스,649,ifrs_CashFlowsFromUsedInOperatingActivities,영업활동현금흐름,"-1,568,988,010"
5,265520,2021,사업보고서,AP시스템,292,ifrs_CashFlowsFromUsedInOperatingActivities,영업활동현금흐름,"63,249,106,032"
...,...,...,...,...,...,...,...,...
14861,010240,2022,1분기보고서,흥국,292,ifrs_CashFlowsFromUsedInFinancingActivities,재무활동현금흐름,"-2,309,643,228"
14862,189980,2022,1분기보고서,흥국에프엔비,112,ifrs_CashFlowsFromUsedInFinancingActivities,재무활동현금흐름,"-75,964,141"
14863,003280,2022,1분기보고서,흥아해운,501,ifrs_CashFlowsFromUsedInFinancingActivities,재무활동현금흐름,"-3,544,038,131"
14864,037440,2022,1분기보고서,희림,721,ifrs_CashFlowsFromUsedInFinancingActivities,재무활동현금흐름,"-2,162,502,506"


In [83]:
현금흐름샘플['항목명'] = 현금흐름샘플.apply(lambda x: '영업활동현금흐름' if '영업활동' in x['항목명'] else x['항목명'], axis=1)
현금흐름샘플['항목명'] = 현금흐름샘플.apply(lambda x: '투자활동현금흐름' if '투자활동' in x['항목명'] else x['항목명'], axis=1)
현금흐름샘플['항목명'] = 현금흐름샘플.apply(lambda x: '재무활동현금흐름' if '재무활동' in x['항목명'] else x['항목명'], axis=1)

현금흐름샘플['항목코드'] = 현금흐름샘플.apply(lambda x: 'ifrs_CashFlowsFromUsedInOperatingActivities' if x['항목명'] == '영업활동현금흐름' else x['항목코드'], axis=1)
현금흐름샘플['항목코드'] = 현금흐름샘플.apply(lambda x: 'ifrs_CashFlowsFromUsedInInvestingActivities' if x['항목명'] == '투자활동현금흐름' else x['항목코드'], axis=1)
현금흐름샘플['항목코드'] = 현금흐름샘플.apply(lambda x: 'ifrs_CashFlowsFromUsedInFinancingActivities' if x['항목명'] == '재무활동현금흐름' else x['항목코드'], axis=1)

현금흐름샘플

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documen

,종목코드,사업연도,보고서종류,회사명,업종,항목코드,항목명,당기
0,060310,2021,사업보고서,3S,292,ifrs_CashFlowsFromUsedInOperatingActivities,영업활동현금흐름,"871,088,551"
2,095570,2021,사업보고서,AJ네트웍스,763,ifrs_CashFlowsFromUsedInOperatingActivities,영업활동현금흐름,"-11,318,072,318"
3,006840,2021,사업보고서,AK홀딩스,649,ifrs_CashFlowsFromUsedInOperatingActivities,영업활동현금흐름,"-9,766,237,791"
4,054620,2021,사업보고서,APS홀딩스,649,ifrs_CashFlowsFromUsedInOperatingActivities,영업활동현금흐름,"-1,568,988,010"
5,265520,2021,사업보고서,AP시스템,292,ifrs_CashFlowsFromUsedInOperatingActivities,영업활동현금흐름,"63,249,106,032"
...,...,...,...,...,...,...,...,...
14861,010240,2022,1분기보고서,흥국,292,ifrs_CashFlowsFromUsedInFinancingActivities,재무활동현금흐름,"-2,309,643,228"
14862,189980,2022,1분기보고서,흥국에프엔비,112,ifrs_CashFlowsFromUsedInFinancingActivities,재무활동현금흐름,"-75,964,141"
14863,003280,2022,1분기보고서,흥아해운,501,ifrs_CashFlowsFromUsedInFinancingActivities,재무활동현금흐름,"-3,544,038,131"
14864,037440,2022,1분기보고서,희림,721,ifrs_CashFlowsFromUsedInFinancingActivities,재무활동현금흐름,"-2,162,502,506"


In [84]:
현금흐름샘플 = 현금흐름샘플.drop_duplicates()

In [85]:
현금흐름샘플['항목명'].unique()

array(['영업활동현금흐름', '투자활동현금흐름', '재무활동현금흐름'], dtype=object)

In [86]:
현금흐름샘플['항목코드'].unique()

array(['ifrs_CashFlowsFromUsedInOperatingActivities',
       'ifrs_CashFlowsFromUsedInInvestingActivities',
       'ifrs_CashFlowsFromUsedInFinancingActivities'], dtype=object)

In [87]:
현금흐름일괄 = 현금흐름샘플.drop_duplicates()
현금흐름일괄

,종목코드,사업연도,보고서종류,회사명,업종,항목코드,항목명,당기
0,060310,2021,사업보고서,3S,292,ifrs_CashFlowsFromUsedInOperatingActivities,영업활동현금흐름,"871,088,551"
2,095570,2021,사업보고서,AJ네트웍스,763,ifrs_CashFlowsFromUsedInOperatingActivities,영업활동현금흐름,"-11,318,072,318"
3,006840,2021,사업보고서,AK홀딩스,649,ifrs_CashFlowsFromUsedInOperatingActivities,영업활동현금흐름,"-9,766,237,791"
4,054620,2021,사업보고서,APS홀딩스,649,ifrs_CashFlowsFromUsedInOperatingActivities,영업활동현금흐름,"-1,568,988,010"
5,265520,2021,사업보고서,AP시스템,292,ifrs_CashFlowsFromUsedInOperatingActivities,영업활동현금흐름,"63,249,106,032"
...,...,...,...,...,...,...,...,...
14861,010240,2022,1분기보고서,흥국,292,ifrs_CashFlowsFromUsedInFinancingActivities,재무활동현금흐름,"-2,309,643,228"
14862,189980,2022,1분기보고서,흥국에프엔비,112,ifrs_CashFlowsFromUsedInFinancingActivities,재무활동현금흐름,"-75,964,141"
14863,003280,2022,1분기보고서,흥아해운,501,ifrs_CashFlowsFromUsedInFinancingActivities,재무활동현금흐름,"-3,544,038,131"
14864,037440,2022,1분기보고서,희림,721,ifrs_CashFlowsFromUsedInFinancingActivities,재무활동현금흐름,"-2,162,502,506"


## 손익계산서

In [88]:
손익계산샘플['종목코드'] = 손익계산샘플['종목코드'].apply(lambda x:x[1:-1]) 
mask = (손익계산샘플['항목명'].str.contains('포괄|계속|중단'))|(손익계산샘플['항목코드'].str.contains('continu|Continu'))
손익계산샘플 = 손익계산샘플[~mask]

In [89]:
손익계산샘플['항목명'] = 손익계산샘플.apply(lambda x: '매출액' if x['항목코드'] == 'ifrs_Revenue' else x['항목명'], axis=1)
손익계산샘플['항목명'] = 손익계산샘플.apply(lambda x: '매출액' if x['항목코드'] == 'ifrs-full_Revenue' else x['항목명'], axis=1)
손익계산샘플['항목명'] = 손익계산샘플.apply(lambda x: '매출총이익' if x['항목코드'] == 'ifrs_GrossProfit' else x['항목명'], axis=1)
손익계산샘플['항목명'] = 손익계산샘플.apply(lambda x: '매출총이익' if x['항목코드'] == 'ifrs-full_GrossProfit' else x['항목명'], axis=1)
손익계산샘플['항목명'] = 손익계산샘플.apply(lambda x: '영업이익' if x['항목코드'] == 'dart_OperatingIncomeLoss' else x['항목명'], axis=1)
손익계산샘플['항목명'] = 손익계산샘플.apply(lambda x: '당기순이익' if x['항목코드'] == 'ifrs_ProfitLoss' else x['항목명'], axis=1)
손익계산샘플['항목명'] = 손익계산샘플.apply(lambda x: '당기순이익' if x['항목코드'] == 'ifrs-full_ProfitLoss' else x['항목명'], axis=1)

손익계산샘플['항목명'] = 손익계산샘플.apply(lambda x: '당기순이익(비지배)' if 'ProfitLossAttributableToNoncontrollingInterests' in x['항목코드'] else x['항목명'], axis=1)

손익계산샘플['항목명'] = 손익계산샘플.apply(lambda x: '감가상각비' if x['항목코드'] == 'dart_DepreciationExpense' else x['항목명'], axis=1)
손익계산샘플['항목명'] = 손익계산샘플.apply(lambda x: '법인세비용차감전순이익' if x['항목코드'] == 'ifrs_ProfitLossBeforeTax' else x['항목명'], axis=1)
손익계산샘플['항목명'] = 손익계산샘플.apply(lambda x: '법인세비용차감전순이익' if x['항목코드'] == 'ifrs-full_ProfitLossBeforeTax' else x['항목명'], axis=1)
손익계산샘플['항목명'] = 손익계산샘플.apply(lambda x: '매출액' if x['항목명'] == '수익(매출액)' else x['항목명'], axis=1)
손익계산샘플['항목명'] = 손익계산샘플.apply(lambda x: '매출총이익' if x['항목명'] == '매출총이익(손실)' else x['항목명'], axis=1)
손익계산샘플['항목명'] = 손익계산샘플.apply(lambda x: '영업이익' if x['항목명'] == '영업이익(손실)' else x['항목명'], axis=1)
손익계산샘플['항목명'] = 손익계산샘플.apply(lambda x: '당기순이익' if x['항목명'] == '당기순이익(손실)' else x['항목명'], axis=1)
손익계산샘플['항목명'] = 손익계산샘플.apply(lambda x: '법인세차감전순이익' if x['항목명'] == '법인세차감전순이익(손실)' else x['항목명'], axis=1)
손익계산샘플['항목명'] = 손익계산샘플.apply(lambda x: '법인세차감전순이익' if x['항목명'] == '법인세비용차감전순이익' else x['항목명'], axis=1)


손익계산샘플['항목코드'] = 손익계산샘플.apply(lambda x: 'ifrs_Revenue' if x['항목명'] == '매출액' else x['항목코드'], axis=1)
손익계산샘플['항목코드'] = 손익계산샘플.apply(lambda x: 'ifrs_GrossProfit' if x['항목명'] == '매출총이익' else x['항목코드'], axis=1)
손익계산샘플['항목코드'] = 손익계산샘플.apply(lambda x: 'dart_OperatingIncomeLoss' if x['항목명'] == '영업이익' else x['항목코드'], axis=1)
손익계산샘플['항목코드'] = 손익계산샘플.apply(lambda x: 'ifrs_ProfitLoss' if x['항목명'] == '당기순이익' else x['항목코드'], axis=1)
손익계산샘플['항목코드'] = 손익계산샘플.apply(lambda x: 'ifrs_NoncontrollingInterests' if '비지배' in x['항목명'] else x['항목코드'], axis=1)
손익계산샘플['항목코드'] = 손익계산샘플.apply(lambda x: 'dart_DepreciationExpense' if x['항목명'] == '감가상각비' else x['항목코드'], axis=1)
손익계산샘플['항목코드'] = 손익계산샘플.apply(lambda x: 'ifrs_ProfitLossBeforeTax' if x['항목명'] == '법인세차감전순이익' else x['항목코드'], axis=1)

손익계산샘플['항목명'] = 손익계산샘플.apply(lambda x: '당기순이익(비지배)' if 'ProfitLossAttributableToNoncontrollingInterests' in x['항목코드'] else x['항목명'], axis=1)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documen

In [90]:
손익계산샘플 = 손익계산샘플.drop_duplicates()

In [91]:
손익계산샘플['항목명'].unique()

array(['매출액', '매출총이익', '영업이익', '당기순이익', '당기순이익(비지배)', '비지배지분',
       '비지배지분에 귀속되는 당기순이익', '비지배지분순이익', '감가상각비', '법인세차감전순이익'],
      dtype=object)

In [92]:
손익계산샘플['항목코드'].unique()

array(['ifrs_Revenue', 'ifrs_GrossProfit', 'dart_OperatingIncomeLoss',
       'ifrs_ProfitLoss', 'ifrs_NoncontrollingInterests',
       'dart_DepreciationExpense', 'ifrs_ProfitLossBeforeTax'],
      dtype=object)

In [93]:
손익계산샘플[손익계산샘플['항목명'] == '당기순이익(비지배)']

,종목코드,사업연도,보고서종류,회사명,업종,항목코드,항목명,당기
644,001040,2021,사업보고서,CJ,649,ifrs_NoncontrollingInterests,당기순이익(비지배),"531,987,271,000"
645,000120,2021,사업보고서,CJ대한통운,493,ifrs_NoncontrollingInterests,당기순이익(비지배),"103,537,367,210"
646,000990,2021,사업보고서,DB하이텍,261,ifrs_NoncontrollingInterests,당기순이익(비지배),0
647,000210,2021,사업보고서,DL,649,ifrs_NoncontrollingInterests,당기순이익(비지배),"9,624,006,922"
649,155660,2021,사업보고서,DSR,242,ifrs_NoncontrollingInterests,당기순이익(비지배),"248,957,034"
...,...,...,...,...,...,...,...,...
1760,111110,2022,1분기보고서,호전실업,141,ifrs_NoncontrollingInterests,당기순이익(비지배),"-23,536,148"
1761,008770,2022,1분기보고서,호텔신라,478,ifrs_NoncontrollingInterests,당기순이익(비지배),"-718,844"
1763,298050,2022,1분기보고서,효성첨단소재,205,ifrs_NoncontrollingInterests,당기순이익(비지배),"14,719,015,070"
1764,298020,2022,1분기보고서,효성티앤씨,205,ifrs_NoncontrollingInterests,당기순이익(비지배),"36,723,820,139"


## 포괄손익계산서

In [94]:
포괄손익계산샘플['종목코드'] = 포괄손익계산샘플['종목코드'].apply(lambda x:x[1:-1]) 
mask = (포괄손익계산샘플['항목명'].str.contains('포괄|계속|중단|해외'))|(포괄손익계산샘플['항목코드'].str.contains('Operations|ComprehensiveIncome'))
포괄손익계산샘플 = 포괄손익계산샘플[~mask]

In [95]:
포괄손익계산샘플['항목명'] = 포괄손익계산샘플.apply(lambda x: '매출액' if x['항목코드'] == 'ifrs_Revenue' else x['항목명'], axis=1)
포괄손익계산샘플['항목명'] = 포괄손익계산샘플.apply(lambda x: '매출액' if x['항목코드'] == 'ifrs-full_Revenue' else x['항목명'], axis=1)
포괄손익계산샘플['항목명'] = 포괄손익계산샘플.apply(lambda x: '매출총이익' if x['항목코드'] == 'ifrs_GrossProfit' else x['항목명'], axis=1)
포괄손익계산샘플['항목명'] = 포괄손익계산샘플.apply(lambda x: '매출총이익' if x['항목코드'] == 'ifrs-full_GrossProfit' else x['항목명'], axis=1)
포괄손익계산샘플['항목명'] = 포괄손익계산샘플.apply(lambda x: '영업이익' if x['항목코드'] == 'dart_OperatingIncomeLoss' else x['항목명'], axis=1)
포괄손익계산샘플['항목명'] = 포괄손익계산샘플.apply(lambda x: '당기순이익' if x['항목코드'] == 'ifrs_ProfitLoss' else x['항목명'], axis=1)
포괄손익계산샘플['항목명'] = 포괄손익계산샘플.apply(lambda x: '당기순이익' if x['항목코드'] == 'ifrs-full_ProfitLoss' else x['항목명'], axis=1)

포괄손익계산샘플['항목명'] = 포괄손익계산샘플.apply(lambda x: '당기순이익(비지배)' if 'ProfitLossAttributableToNoncontrollingInterests' in x['항목코드'] else x['항목명'], axis=1)
포괄손익계산샘플['항목명'] = 포괄손익계산샘플.apply(lambda x: '당기순이익(비지배)' if 'ifrs_ProfitLossAttributableToNoncontrollingInterests' in x['항목코드'] else x['항목명'], axis=1)

포괄손익계산샘플['항목명'] = 포괄손익계산샘플.apply(lambda x: '감가상각비' if x['항목코드'] == 'dart_DepreciationExpense' else x['항목명'], axis=1)
포괄손익계산샘플['항목명'] = 포괄손익계산샘플.apply(lambda x: '법인세비용차감전순이익' if x['항목코드'] == 'ifrs_ProfitLossBeforeTax' else x['항목명'], axis=1)
포괄손익계산샘플['항목명'] = 포괄손익계산샘플.apply(lambda x: '법인세비용차감전순이익' if x['항목코드'] == 'ifrs-full_ProfitLossBeforeTax' else x['항목명'], axis=1)
포괄손익계산샘플['항목명'] = 포괄손익계산샘플.apply(lambda x: '매출액' if x['항목명'] == '수익(매출액)' else x['항목명'], axis=1)
포괄손익계산샘플['항목명'] = 포괄손익계산샘플.apply(lambda x: '매출총이익' if x['항목명'] == '매출총이익(손실)' else x['항목명'], axis=1)
포괄손익계산샘플['항목명'] = 포괄손익계산샘플.apply(lambda x: '영업이익' if x['항목명'] == '영업이익(손실)' else x['항목명'], axis=1)
포괄손익계산샘플['항목명'] = 포괄손익계산샘플.apply(lambda x: '당기순이익' if x['항목명'] == '당기순이익(손실)' else x['항목명'], axis=1)
포괄손익계산샘플['항목명'] = 포괄손익계산샘플.apply(lambda x: '법인세차감전순이익' if x['항목명'] == '법인세차감전순이익(손실)' else x['항목명'], axis=1)
포괄손익계산샘플['항목명'] = 포괄손익계산샘플.apply(lambda x: '법인세차감전순이익' if x['항목명'] == '법인세비용차감전순이익' else x['항목명'], axis=1)

포괄손익계산샘플['항목명'] = 포괄손익계산샘플.apply(lambda x: '당기순이익(비지배)' if '비지배' in x['항목명'] else x['항목명'], axis=1)
포괄손익계산샘플['항목코드'] = 포괄손익계산샘플.apply(lambda x: 'ProfitLossAttributableToNoncontrollingInterests' if x['항목명'] == '당기순이익(비지배)' else x['항목코드'], axis=1)

포괄손익계산샘플['항목코드'] = 포괄손익계산샘플.apply(lambda x: 'ifrs_Revenue' if x['항목명'] == '매출액' else x['항목코드'], axis=1)
포괄손익계산샘플['항목코드'] = 포괄손익계산샘플.apply(lambda x: 'ifrs_GrossProfit' if x['항목명'] == '매출총이익' else x['항목코드'], axis=1)
포괄손익계산샘플['항목코드'] = 포괄손익계산샘플.apply(lambda x: 'dart_OperatingIncomeLoss' if x['항목명'] == '영업이익' else x['항목코드'], axis=1)
포괄손익계산샘플['항목코드'] = 포괄손익계산샘플.apply(lambda x: 'ifrs_ProfitLoss' if x['항목명'] == '당기순이익' else x['항목코드'], axis=1)
포괄손익계산샘플['항목코드'] = 포괄손익계산샘플.apply(lambda x: 'ifrs_ProfitLossAttributableToNoncontrollingInterests' if '비지배' in x['항목명'] else x['항목코드'], axis=1)
포괄손익계산샘플['항목코드'] = 포괄손익계산샘플.apply(lambda x: 'dart_DepreciationExpense' if x['항목명'] == '감가상각비' else x['항목코드'], axis=1)
포괄손익계산샘플['항목코드'] = 포괄손익계산샘플.apply(lambda x: 'ifrs_ProfitLossBeforeTax' if x['항목명'] == '법인세차감전순이익' else x['항목코드'], axis=1)

포괄손익계산샘플

,종목코드,사업연도,보고서종류,회사명,업종,항목코드,항목명,당기
0,060310,2021,사업보고서,3S,292,ifrs_Revenue,매출액,"23,477,908,383"
1,095570,2021,사업보고서,AJ네트웍스,763,ifrs_Revenue,매출액,"1,026,749,457,003"
2,006840,2021,사업보고서,AK홀딩스,649,ifrs_Revenue,매출액,"3,167,699,652,090"
3,054620,2021,사업보고서,APS홀딩스,649,ifrs_Revenue,매출액,"39,415,521,257"
4,265520,2021,사업보고서,AP시스템,292,ifrs_Revenue,매출액,"528,740,970,343"
...,...,...,...,...,...,...,...,...
22625,010240,2022,1분기보고서,흥국,292,ifrs_ProfitLossBeforeTax,법인세차감전순이익,"3,353,383,900"
22626,189980,2022,1분기보고서,흥국에프엔비,112,ifrs_ProfitLossBeforeTax,법인세차감전순이익,"1,236,152,104"
22627,003280,2022,1분기보고서,흥아해운,501,ifrs_ProfitLossBeforeTax,법인세차감전순이익,"-371,037,287"
22628,037440,2022,1분기보고서,희림,721,ifrs_ProfitLossBeforeTax,법인세차감전순이익,"1,745,202,772"


In [96]:
포괄손익계산샘플['항목명'] = 포괄손익계산샘플.apply(lambda x: '당기순이익(비지배)' if '비지배' in x['항목명'] else x['항목명'], axis=1)

In [97]:
포괄손익계산샘플 = 포괄손익계산샘플.drop_duplicates()
포괄손익계산샘플

,종목코드,사업연도,보고서종류,회사명,업종,항목코드,항목명,당기
0,060310,2021,사업보고서,3S,292,ifrs_Revenue,매출액,"23,477,908,383"
1,095570,2021,사업보고서,AJ네트웍스,763,ifrs_Revenue,매출액,"1,026,749,457,003"
2,006840,2021,사업보고서,AK홀딩스,649,ifrs_Revenue,매출액,"3,167,699,652,090"
3,054620,2021,사업보고서,APS홀딩스,649,ifrs_Revenue,매출액,"39,415,521,257"
4,265520,2021,사업보고서,AP시스템,292,ifrs_Revenue,매출액,"528,740,970,343"
...,...,...,...,...,...,...,...,...
22625,010240,2022,1분기보고서,흥국,292,ifrs_ProfitLossBeforeTax,법인세차감전순이익,"3,353,383,900"
22626,189980,2022,1분기보고서,흥국에프엔비,112,ifrs_ProfitLossBeforeTax,법인세차감전순이익,"1,236,152,104"
22627,003280,2022,1분기보고서,흥아해운,501,ifrs_ProfitLossBeforeTax,법인세차감전순이익,"-371,037,287"
22628,037440,2022,1분기보고서,희림,721,ifrs_ProfitLossBeforeTax,법인세차감전순이익,"1,745,202,772"


In [98]:
len(포괄손익계산샘플['종목코드'].unique())

1846

In [99]:
포괄손익계산샘플['항목코드'].unique()

array(['ifrs_Revenue', 'ifrs_GrossProfit', 'dart_OperatingIncomeLoss',
       'ifrs_ProfitLoss',
       'ifrs_ProfitLossAttributableToNoncontrollingInterests',
       'dart_DepreciationExpense', 'ifrs_ProfitLossBeforeTax'],
      dtype=object)

In [100]:
len(포괄손익계산샘플['항목코드'].unique())

7

In [101]:
포괄손익계산샘플['항목명'].unique()

array(['매출액', '매출총이익', '영업이익', '당기순이익', '당기순이익(비지배)', '감가상각비',
       '법인세차감전순이익'], dtype=object)

## 재무상태표

In [102]:
재무상태샘플['종목코드'] = 재무상태샘플['종목코드'].apply(lambda x:x[1:-1])
mask = (재무상태샘플['항목명'].str.contains('손익'))|(재무상태샘플['항목명'].isin(['비지배지분 부채', '비지배주주지분부채', '비지배지분금융부채', '유동비지배지분부채', '비지배주주지분 부채', '비지배지분 비유동부채', '비지배지분 유동부채']))
재무상태샘플 = 재무상태샘플[~mask]

In [103]:
재무상태샘플['항목명'] = 재무상태샘플.apply(lambda x: '자산총계' if x['항목코드'] == 'ifrs_Assets' else x['항목명'], axis=1)
재무상태샘플['항목명'] = 재무상태샘플.apply(lambda x: '자산총계' if x['항목코드'] == 'ifrs-full_Assets' else x['항목명'], axis=1)
재무상태샘플['항목명'] = 재무상태샘플.apply(lambda x: '부채총계' if x['항목코드'] == 'ifrs_Liabilities' else x['항목명'], axis=1)
재무상태샘플['항목명'] = 재무상태샘플.apply(lambda x: '부채총계' if x['항목코드'] == 'ifrs-full_Liabilities' else x['항목명'], axis=1)
재무상태샘플['항목명'] = 재무상태샘플.apply(lambda x: '자본총계' if x['항목코드'] == 'ifrs_Equity' else x['항목명'], axis=1)
재무상태샘플['항목명'] = 재무상태샘플.apply(lambda x: '자본총계' if x['항목코드'] == 'ifrs-full_Equity' else x['항목명'], axis=1)
재무상태샘플['항목명'] = 재무상태샘플.apply(lambda x: '자본총계(비지배)' if x['항목코드'] == 'ifrs_NoncontrollingInterests' else x['항목명'], axis=1)
재무상태샘플['항목명'] = 재무상태샘플.apply(lambda x: '자본총계(비지배)' if x['항목코드'] == 'ifrs-full_NoncontrollingInterests' else x['항목명'], axis=1)
재무상태샘플['항목명'] = 재무상태샘플.apply(lambda x: '유동자산' if x['항목코드'] == 'ifrs_CurrentAssets' else x['항목명'], axis=1)
재무상태샘플['항목명'] = 재무상태샘플.apply(lambda x: '유동자산' if x['항목코드'] == 'ifrs-full_CurrentAssets' else x['항목명'], axis=1)
재무상태샘플['항목명'] = 재무상태샘플.apply(lambda x: '현금및현금성자산' if x['항목코드'] == 'ifrs_CashAndCashEquivalents' else x['항목명'], axis=1)
재무상태샘플['항목명'] = 재무상태샘플.apply(lambda x: '현금및현금성자산' if x['항목코드'] == 'ifrs-full_CashAndCashEquivalents' else x['항목명'], axis=1)
재무상태샘플['항목명'] = 재무상태샘플.apply(lambda x: '자산총계' if x['항목명'] == '자산 총계' else x['항목명'], axis=1)
재무상태샘플['항목명'] = 재무상태샘플.apply(lambda x: '자본총계' if x['항목명'] == '자본 총계' else x['항목명'], axis=1)
재무상태샘플['항목명'] = 재무상태샘플.apply(lambda x: '자본총계(비지배)' if x['항목코드'] == 'ifrs_NoncontrollingInterests' else x['항목명'], axis=1)
재무상태샘플['항목명'] = 재무상태샘플.apply(lambda x: '부채총계' if x['항목명'] == '부채 총계' else x['항목명'], axis=1)
재무상태샘플['항목명'] = 재무상태샘플.apply(lambda x: '유동자산' if x['항목명'] == '유동 자산' else x['항목명'], axis=1)
재무상태샘플['항목명'] = 재무상태샘플.apply(lambda x: '현금및현금성자산' if x['항목명'] == '현금' else x['항목명'], axis=1)
재무상태샘플['항목명'] = 재무상태샘플.apply(lambda x: '현금및현금성자산' if x['항목명'] == '현금성자산' else x['항목명'], axis=1)
재무상태샘플['항목명'] = 재무상태샘플.apply(lambda x: '현금및현금성자산' if x['항목명'] == '기타현금및현금성자산' else x['항목명'], axis=1)
재무상태샘플['항목명'] = 재무상태샘플.apply(lambda x: '현금및현금성자산' if x['항목명'] == '현금 및 현금성자산' else x['항목명'], axis=1)
재무상태샘플['항목명'] = 재무상태샘플.apply(lambda x: '현금및현금성자산' if x['항목명'] == '현금및현금성자산-증권업' else x['항목명'], axis=1)

재무상태샘플['항목코드'] = 재무상태샘플.apply(lambda x: 'ifrs_Assets' if x['항목명'] == '자산총계' else x['항목코드'], axis=1)
재무상태샘플['항목코드'] = 재무상태샘플.apply(lambda x: 'ifrs_Liabilities' if x['항목명'] == '부채총계' else x['항목코드'], axis=1)
재무상태샘플['항목코드'] = 재무상태샘플.apply(lambda x: 'ifrs_Equity' if x['항목명'] == '자본총계' else x['항목코드'], axis=1)
재무상태샘플['항목코드'] = 재무상태샘플.apply(lambda x: 'ifrs_CurrentAssets' if x['항목명'] == '유동자산' else x['항목코드'], axis=1)
재무상태샘플['항목코드'] = 재무상태샘플.apply(lambda x: 'ifrs_CashAndCashEquivalents' if x['항목명'] == '현금및현금성자산' else x['항목코드'], axis=1)
재무상태샘플['항목코드'] = 재무상태샘플.apply(lambda x: 'ifrs_NoncontrollingInterests' if '비지배' in x['항목명'] else x['항목코드'], axis=1)
재무상태샘플['항목명'] = 재무상태샘플.apply(lambda x: '자본총계(비지배)' if x['항목코드'] == 'ifrs_NoncontrollingInterests' else x['항목명'], axis=1)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documen

In [104]:
재무상태샘플

,종목코드,사업연도,보고서종류,회사명,업종,항목코드,항목명,당기
0,060310,2021,사업보고서,3S,292,ifrs_Assets,자산총계,"54,614,752,349"
1,095570,2021,사업보고서,AJ네트웍스,763,ifrs_Assets,자산총계,"1,355,044,449,955"
2,006840,2021,사업보고서,AK홀딩스,649,ifrs_Assets,자산총계,"4,548,736,656,905"
3,054620,2021,사업보고서,APS홀딩스,649,ifrs_Assets,자산총계,"337,092,577,346"
4,265520,2021,사업보고서,AP시스템,292,ifrs_Assets,자산총계,"428,720,088,928"
...,...,...,...,...,...,...,...,...
21567,010240,2022,1분기보고서,흥국,292,ifrs_CashAndCashEquivalents,현금및현금성자산,"16,028,579,778"
21568,189980,2022,1분기보고서,흥국에프엔비,112,ifrs_CashAndCashEquivalents,현금및현금성자산,"17,648,654,115"
21569,003280,2022,1분기보고서,흥아해운,501,ifrs_CashAndCashEquivalents,현금및현금성자산,"48,758,422,117"
21570,037440,2022,1분기보고서,희림,721,ifrs_CashAndCashEquivalents,현금및현금성자산,"26,045,207,278"


In [105]:
재무상태샘플['항목명'].unique()

array(['자산총계', '자본총계', '자본총계(비지배)', '부채총계', '유동자산', '현금및현금성자산'],
      dtype=object)

In [106]:
재무상태샘플['항목코드'].unique()

array(['ifrs_Assets', 'ifrs_Equity', 'ifrs_NoncontrollingInterests',
       'ifrs_Liabilities', 'ifrs_CurrentAssets',
       'ifrs_CashAndCashEquivalents'], dtype=object)

# 저장

In [107]:
import datetime as dt
x = dt.datetime.now()
x.year, x.month, x.day

(2022, 7, 5)

In [108]:
현금흐름샘플.to_csv('/content/drive/MyDrive/22캡스톤디자인/재무일괄/업데이트/(21-4, 22-1)/현금흐름일괄.csv')
재무상태샘플.to_csv('/content/drive/MyDrive/22캡스톤디자인/재무일괄/업데이트/(21-4, 22-1)/재무상태표일괄.csv')
포괄손익계산샘플.to_csv('/content/drive/MyDrive/22캡스톤디자인/재무일괄/업데이트/(21-4, 22-1)/포괄손익일괄.csv')
손익계산샘플.to_csv('/content/drive/MyDrive/22캡스톤디자인/재무일괄/업데이트/(21-4, 22-1)/손익일괄.csv')

# 통합

In [109]:
재무상태표일괄 = pd.read_csv('/content/drive/MyDrive/22캡스톤디자인/재무일괄/업데이트/(21-4, 22-1)/재무상태표일괄.csv',index_col = 0,  dtype={'종목코드':'str'})
손익일괄 = pd.read_csv('/content/drive/MyDrive/22캡스톤디자인/재무일괄/업데이트/(21-4, 22-1)/손익일괄.csv',index_col = 0,  dtype={'종목코드':'str'})
포괄손익일괄 = pd.read_csv('/content/drive/MyDrive/22캡스톤디자인/재무일괄/업데이트/(21-4, 22-1)/포괄손익일괄.csv',index_col = 0,  dtype={'종목코드':'str'})
현금흐름일괄 = pd.read_csv('/content/drive/MyDrive/22캡스톤디자인/재무일괄/업데이트/(21-4, 22-1)/현금흐름일괄.csv',index_col = 0,  dtype={'종목코드':'str'})

In [110]:
현금흐름일괄

,종목코드,사업연도,보고서종류,회사명,업종,항목코드,항목명,당기
0,060310,2021,사업보고서,3S,292,ifrs_CashFlowsFromUsedInOperatingActivities,영업활동현금흐름,"871,088,551"
2,095570,2021,사업보고서,AJ네트웍스,763,ifrs_CashFlowsFromUsedInOperatingActivities,영업활동현금흐름,"-11,318,072,318"
3,006840,2021,사업보고서,AK홀딩스,649,ifrs_CashFlowsFromUsedInOperatingActivities,영업활동현금흐름,"-9,766,237,791"
4,054620,2021,사업보고서,APS홀딩스,649,ifrs_CashFlowsFromUsedInOperatingActivities,영업활동현금흐름,"-1,568,988,010"
5,265520,2021,사업보고서,AP시스템,292,ifrs_CashFlowsFromUsedInOperatingActivities,영업활동현금흐름,"63,249,106,032"
...,...,...,...,...,...,...,...,...
14861,010240,2022,1분기보고서,흥국,292,ifrs_CashFlowsFromUsedInFinancingActivities,재무활동현금흐름,"-2,309,643,228"
14862,189980,2022,1분기보고서,흥국에프엔비,112,ifrs_CashFlowsFromUsedInFinancingActivities,재무활동현금흐름,"-75,964,141"
14863,003280,2022,1분기보고서,흥아해운,501,ifrs_CashFlowsFromUsedInFinancingActivities,재무활동현금흐름,"-3,544,038,131"
14864,037440,2022,1분기보고서,희림,721,ifrs_CashFlowsFromUsedInFinancingActivities,재무활동현금흐름,"-2,162,502,506"


In [111]:
재무상태표일괄

,종목코드,사업연도,보고서종류,회사명,업종,항목코드,항목명,당기
0,060310,2021,사업보고서,3S,292,ifrs_Assets,자산총계,"54,614,752,349"
1,095570,2021,사업보고서,AJ네트웍스,763,ifrs_Assets,자산총계,"1,355,044,449,955"
2,006840,2021,사업보고서,AK홀딩스,649,ifrs_Assets,자산총계,"4,548,736,656,905"
3,054620,2021,사업보고서,APS홀딩스,649,ifrs_Assets,자산총계,"337,092,577,346"
4,265520,2021,사업보고서,AP시스템,292,ifrs_Assets,자산총계,"428,720,088,928"
...,...,...,...,...,...,...,...,...
21567,010240,2022,1분기보고서,흥국,292,ifrs_CashAndCashEquivalents,현금및현금성자산,"16,028,579,778"
21568,189980,2022,1분기보고서,흥국에프엔비,112,ifrs_CashAndCashEquivalents,현금및현금성자산,"17,648,654,115"
21569,003280,2022,1분기보고서,흥아해운,501,ifrs_CashAndCashEquivalents,현금및현금성자산,"48,758,422,117"
21570,037440,2022,1분기보고서,희림,721,ifrs_CashAndCashEquivalents,현금및현금성자산,"26,045,207,278"


In [112]:
손익일괄

,종목코드,사업연도,보고서종류,회사명,업종,항목코드,항목명,당기
0,001040,2021,사업보고서,CJ,649,ifrs_Revenue,매출액,"34,484,026,554,000"
1,000120,2021,사업보고서,CJ대한통운,493,ifrs_Revenue,매출액,"11,343,651,787,492"
2,000990,2021,사업보고서,DB하이텍,261,ifrs_Revenue,매출액,"1,214,682,040,178"
3,000210,2021,사업보고서,DL,649,ifrs_Revenue,매출액,"2,385,420,753,496"
4,375500,2021,사업보고서,DL이앤씨,412,ifrs_Revenue,매출액,"7,631,650,898,582"
...,...,...,...,...,...,...,...,...
1930,111110,2022,1분기보고서,호전실업,141,ifrs_ProfitLossBeforeTax,법인세차감전순이익,"3,074,888,641"
1931,008770,2022,1분기보고서,호텔신라,478,ifrs_ProfitLossBeforeTax,법인세차감전순이익,"-7,795,875,667"
1933,298050,2022,1분기보고서,효성첨단소재,205,ifrs_ProfitLossBeforeTax,법인세차감전순이익,"93,763,702,404"
1934,298020,2022,1분기보고서,효성티앤씨,205,ifrs_ProfitLossBeforeTax,법인세차감전순이익,"159,571,477,299"


In [113]:
포괄손익일괄

,종목코드,사업연도,보고서종류,회사명,업종,항목코드,항목명,당기
0,060310,2021,사업보고서,3S,292,ifrs_Revenue,매출액,"23,477,908,383"
1,095570,2021,사업보고서,AJ네트웍스,763,ifrs_Revenue,매출액,"1,026,749,457,003"
2,006840,2021,사업보고서,AK홀딩스,649,ifrs_Revenue,매출액,"3,167,699,652,090"
3,054620,2021,사업보고서,APS홀딩스,649,ifrs_Revenue,매출액,"39,415,521,257"
4,265520,2021,사업보고서,AP시스템,292,ifrs_Revenue,매출액,"528,740,970,343"
...,...,...,...,...,...,...,...,...
22625,010240,2022,1분기보고서,흥국,292,ifrs_ProfitLossBeforeTax,법인세차감전순이익,"3,353,383,900"
22626,189980,2022,1분기보고서,흥국에프엔비,112,ifrs_ProfitLossBeforeTax,법인세차감전순이익,"1,236,152,104"
22627,003280,2022,1분기보고서,흥아해운,501,ifrs_ProfitLossBeforeTax,법인세차감전순이익,"-371,037,287"
22628,037440,2022,1분기보고서,희림,721,ifrs_ProfitLossBeforeTax,법인세차감전순이익,"1,745,202,772"


In [114]:
print(f"현금흐름: {len(현금흐름일괄['종목코드'].unique())}")
print(f"손익계산서: {len(손익일괄['종목코드'].unique())}")
print(f"포괄손익계산서: {len(포괄손익일괄['종목코드'].unique())}")
print(f"재무상태표: {len(재무상태표일괄['종목코드'].unique())}")

현금흐름: 1851
손익계산서: 167
포괄손익계산서: 1846
재무상태표: 1851


In [115]:
통합재무정보 = pd.concat([재무상태표일괄, 포괄손익일괄, 손익일괄, 현금흐름일괄]).drop_duplicates()

In [116]:
len(통합재무정보['종목코드'].unique())

1851

In [117]:
통합재무정보['항목코드'].unique()

array(['ifrs_Assets', 'ifrs_Equity', 'ifrs_NoncontrollingInterests',
       'ifrs_Liabilities', 'ifrs_CurrentAssets',
       'ifrs_CashAndCashEquivalents', 'ifrs_Revenue', 'ifrs_GrossProfit',
       'dart_OperatingIncomeLoss', 'ifrs_ProfitLoss',
       'ifrs_ProfitLossAttributableToNoncontrollingInterests',
       'dart_DepreciationExpense', 'ifrs_ProfitLossBeforeTax',
       'ifrs_CashFlowsFromUsedInOperatingActivities',
       'ifrs_CashFlowsFromUsedInInvestingActivities',
       'ifrs_CashFlowsFromUsedInFinancingActivities'], dtype=object)

In [118]:
통합재무정보[통합재무정보['항목코드'].isin(['entity00117708_udf_IS_2020313135558544_udf_IS_2020313135518991_Revenue',
       'entity00117708_udf_IS_202031313565249_udf_IS_2020313135518991_Revenue', 'dart_RevenueFromServices'])]

,종목코드,사업연도,보고서종류,회사명,업종,항목코드,항목명,당기


In [119]:
통합재무정보[통합재무정보['항목코드'].isin(['dart_RevenueFromConstruction'])]

,종목코드,사업연도,보고서종류,회사명,업종,항목코드,항목명,당기


In [120]:
통합재무정보['항목명'].unique()

array(['자산총계', '자본총계', '자본총계(비지배)', '부채총계', '유동자산', '현금및현금성자산', '매출액',
       '매출총이익', '영업이익', '당기순이익', '당기순이익(비지배)', '감가상각비', '법인세차감전순이익',
       '비지배지분', '비지배지분에 귀속되는 당기순이익', '비지배지분순이익', '영업활동현금흐름', '투자활동현금흐름',
       '재무활동현금흐름'], dtype=object)

In [121]:
통합재무정보

,종목코드,사업연도,보고서종류,회사명,업종,항목코드,항목명,당기
0,060310,2021,사업보고서,3S,292,ifrs_Assets,자산총계,"54,614,752,349"
1,095570,2021,사업보고서,AJ네트웍스,763,ifrs_Assets,자산총계,"1,355,044,449,955"
2,006840,2021,사업보고서,AK홀딩스,649,ifrs_Assets,자산총계,"4,548,736,656,905"
3,054620,2021,사업보고서,APS홀딩스,649,ifrs_Assets,자산총계,"337,092,577,346"
4,265520,2021,사업보고서,AP시스템,292,ifrs_Assets,자산총계,"428,720,088,928"
...,...,...,...,...,...,...,...,...
14861,010240,2022,1분기보고서,흥국,292,ifrs_CashFlowsFromUsedInFinancingActivities,재무활동현금흐름,"-2,309,643,228"
14862,189980,2022,1분기보고서,흥국에프엔비,112,ifrs_CashFlowsFromUsedInFinancingActivities,재무활동현금흐름,"-75,964,141"
14863,003280,2022,1분기보고서,흥아해운,501,ifrs_CashFlowsFromUsedInFinancingActivities,재무활동현금흐름,"-3,544,038,131"
14864,037440,2022,1분기보고서,희림,721,ifrs_CashFlowsFromUsedInFinancingActivities,재무활동현금흐름,"-2,162,502,506"


In [122]:
df = pd.DataFrame(
    columns=["종목코드", "사업연도", "보고서종류", "자산총계", "자본총계", "자본총계(비지배)", "부채총계", "유동자산", "당기순이익", "당기순이익(비지배)", "매출액", "현금및현금성자산", "감가상각비", "영업이익", "매출총이익"])

사업연도 = 통합재무정보['사업연도'].unique().tolist()
종목코드리스트 = 통합재무정보['종목코드'].unique().tolist()
보고서종류 = 통합재무정보['보고서종류'].unique().tolist()

for 종목코드 in tqdm(종목코드리스트):
    for 연도 in 사업연도:
        now = 통합재무정보[통합재무정보['종목코드'] == 종목코드]
        then = now[now['사업연도'] == 연도]
        for 보고서 in 보고서종류:
            if not then[then['보고서종류'] == 보고서].empty:
                자산총계 = 자본총계 = 자본총계_비지배 = 부채총계 = 유동자산 = 당기순이익 = 당기순이익_비지배 = 매출액 = 현금및현금성자산 = 감가상각비 = 영업이익 = 매출총이익 = 0
                temp = then[then['보고서종류'] == 보고서]

                if len(temp[temp['항목명'] == '자산총계']) != 0:
                    자산총계 = temp[temp['항목명'] == '자산총계'].iloc[0]['당기']

                if len(temp[temp['항목명'] == '자본총계']) != 0:
                    자본총계 = temp[temp['항목명'] == '자본총계'].iloc[0]['당기']

                if len(temp[temp['항목명'] == '자본총계(비지배)']) != 0:
                    자본총계_비지배 = temp[temp['항목명'] == '자본총계(비지배)'].iloc[0]['당기']

                if len(temp[temp['항목명'] == '부채총계']) != 0:
                    부채총계 = temp[temp['항목명'] == '부채총계'].iloc[0]['당기']

                if len(temp[temp['항목명'] == '유동자산']) != 0:
                    유동자산 = temp[temp['항목명'] == '유동자산'].iloc[0]['당기']

                if len(temp[temp['항목명'] == '당기순이익']) != 0:
                    당기순이익 = temp[temp['항목명'] == '당기순이익'].iloc[0]['당기']

                if len(temp[temp['항목명'] == '당기순이익(비지배)']) != 0:
                    당기순이익_비지배 = temp[temp['항목명'] == '당기순이익(비지배)'].iloc[0]['당기']

                if len(temp[temp['항목명'] == '매출액']) != 0:
                    매출액 = temp[temp['항목명'] == '매출액'].iloc[0]['당기']

                if len(temp[temp['항목명'] == '현금및현금성자산']) != 0:
                    현금및현금성자산 = temp[temp['항목명'] == '현금및현금성자산'].iloc[0]['당기']

                if len(temp[temp['항목명'] == '영업이익']) != 0:
                    영업이익 = temp[temp['항목명'] == '영업이익'].iloc[0]['당기']

                if len(temp[temp['항목명'] == '매출총이익']) != 0:
                    매출총이익 = temp[temp['항목명'] == '매출총이익'].iloc[0]['당기']
                    
                if len(temp[temp['항목명'] == '감가상각비']) != 0:
                    감가상각비 = temp[temp['항목명'] == '감가상각비'].iloc[0]['당기']

                row = {
                    '종목코드': 종목코드,
                    '사업연도': 연도,
                    '보고서종류': 보고서,
                    '자산총계': 자산총계,
                    '자본총계': 자본총계,
                    '자본총계(비지배)': 자본총계_비지배,
                    '부채총계': 부채총계,
                    '유동자산': 유동자산,
                    '당기순이익': 당기순이익,
                    '당기순이익(비지배)': 당기순이익_비지배,
                    '매출액': 매출액,
                    '현금및현금성자산': 현금및현금성자산,
                    '영업이익': 영업이익,
                    '매출총이익': 매출총이익,
                    '감가상각비': 감가상각비
                }
                df = df.append(row, ignore_index=True)


100%|██████████| 1851/1851 [02:26<00:00, 12.60it/s]


In [123]:
df = df.fillna(0)
df

,종목코드,사업연도,보고서종류,자산총계,자본총계,자본총계(비지배),부채총계,유동자산,당기순이익,당기순이익(비지배),매출액,현금및현금성자산,감가상각비,영업이익,매출총이익
0,060310,2021,사업보고서,"54,614,752,349","36,546,324,438","351,168,349","18,068,427,911","18,070,667,851","695,334,883",0,"23,477,908,383","5,875,441,002",0,"-1,408,265,950","1,475,485,938"
1,095570,2021,사업보고서,"1,355,044,449,955","362,510,088,118","1,009,199,717","992,534,361,837","273,912,905,377","76,737,133,075",0,"1,026,749,457,003","85,232,019,813",0,"45,109,542,821",0
2,095570,2022,1분기보고서,"1,367,232,172,184","365,041,242,289","1,046,519,844","1,002,190,929,895","261,873,288,225","10,854,923,388",0,"295,803,640,038","70,572,059,051",0,"19,242,745,659",0
3,006840,2021,사업보고서,"4,548,736,656,905","1,202,359,271,295","602,777,816,403","3,346,377,385,610","1,352,749,114,609","-199,326,056,742","-87,989,450,334","3,167,699,652,090","515,233,699,800",0,"-165,714,944,091","506,430,802,191"
4,006840,2022,1분기보고서,"4,536,502,551,937","1,138,968,629,096","564,728,815,439","3,397,533,922,841","1,365,378,389,947","-48,038,859,624","-24,888,298,635","819,839,856,194","453,619,457,602",0,"-46,391,573,226","122,886,853,703"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3620,368770,2022,1분기보고서,"33,985,155,315","24,767,367,757",0,"9,217,787,558","26,774,815,559","-3,539,090,865",0,"2,792,575,504","13,471,604,472",0,"906,374","1,128,227,561"
3621,377220,2022,1분기보고서,"101,349,710,812","88,872,591,392","916,691,458","12,477,119,420","74,068,874,765","3,482,778,971","-63,308,542","30,126,245,329","2,218,507,004",0,"4,404,131,203","21,475,567,016"
3622,373200,2022,1분기보고서,"17,822,482,974","14,352,229,960","475,725","3,470,253,014","15,633,799,357","-2,077,081,731","-524,275","3,875,711,742","2,596,355,737",0,"439,150,289","1,707,619,646"
3623,220180,2022,1분기보고서,"80,356,551,043","66,789,858,679","3,048,316,618","13,566,692,364","61,794,223,627","962,629,007",0,"3,804,231,879","36,675,723,731",0,"-480,570,405","1,040,697,697"


In [124]:
df.to_csv('/content/drive/MyDrive/22캡스톤디자인/재무일괄/업데이트/(21-4, 22-1)/21년_사업보고서_22년_1분기.csv')

In [125]:
df.describe()

,사업연도
count,3625.000000
mean,2021.495724
std,0.500051
min,2021.000000
25%,2021.000000
50%,2021.000000
75%,2022.000000
max,2022.000000


# 재검

In [126]:
매출액누락 = df[(df['매출액'] == '0')|(df['매출액'] == 0)]['종목코드'].unique().tolist()
영업이익누락 = df[(df['영업이익'] == '0')|(df['영업이익'] == 0)]['종목코드'].unique().tolist()
매출총이익누락 = df[(df['매출총이익'] == '0')|(df['매출총이익'] == 0)]['종목코드'].unique().tolist()
당기순이익누락 = df[(df['당기순이익'] == '0')|(df['당기순이익'] == 0)]['종목코드'].unique().tolist()

전체누락 = list(set(매출액누락 + 영업이익누락 + 매출총이익누락 + 당기순이익누락))

In [127]:
print(f"매출액누락: {len(매출액누락)}")
print(f"영업이익누락: {len(영업이익누락)}")
print(f"매출총이익누락: {len(매출총이익누락)}")
print(f"당기순이익누락: {len(당기순이익누락)}")
print(f"전체누락: {len(전체누락)}")

매출액누락: 56
영업이익누락: 5
매출총이익누락: 127
당기순이익누락: 33
전체누락: 180


In [128]:
df[df['종목코드'].isin(전체누락)]

,종목코드,사업연도,보고서종류,자산총계,자본총계,자본총계(비지배),부채총계,유동자산,당기순이익,당기순이익(비지배),매출액,현금및현금성자산,감가상각비,영업이익,매출총이익
1,095570,2021,사업보고서,"1,355,044,449,955","362,510,088,118","1,009,199,717","992,534,361,837","273,912,905,377","76,737,133,075",0,"1,026,749,457,003","85,232,019,813",0,"45,109,542,821",0
2,095570,2022,1분기보고서,"1,367,232,172,184","365,041,242,289","1,046,519,844","1,002,190,929,895","261,873,288,225","10,854,923,388",0,"295,803,640,038","70,572,059,051",0,"19,242,745,659",0
21,079160,2021,사업보고서,"3,785,492,375,609","301,285,367,542","76,037,441,247","3,484,207,008,067","620,645,630,707","-338,788,006,843","-58,581,055,356","736,315,366,659","338,264,553,232",0,"-241,445,734,017",0
22,079160,2022,1분기보고서,"3,685,788,220,491","180,434,150,098","62,165,563,543","3,505,354,070,393","560,940,437,995","-111,410,097,942","-12,032,737,629","223,264,235,642","272,923,052,114",0,"-54,943,806,015",0
33,023460,2021,사업보고서,"629,691,378,583","131,479,551,617","-903,033,188","498,211,826,966",0,"1,041,966,333",0,"302,353,800,668","33,396,938,013",0,"-1,843,796,817",0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3584,084110,2021,사업보고서,"1,283,747,711,421","851,120,463,935","313,282,303,178","432,627,247,486","618,519,429,354","32,070,878,938","28,076,338,092","579,936,549,949","218,400,053,151",0,"73,342,274,601",0
3585,084110,2022,1분기보고서,"1,297,493,815,576","848,369,337,947","309,606,357,720","449,124,477,629","606,187,551,995","12,803,243,152","7,215,696,097","159,673,252,948","201,416,709,755",0,"20,673,785,662",0
3600,238490,2021,사업보고서,"84,303,020,746","73,916,234,855",0,"10,386,785,891","33,642,552,356","-441,392,436",0,0,"15,509,130,412",0,"-4,371,186,993",0
3601,238490,2022,1분기보고서,"81,851,560,548","70,097,989,109",0,"11,753,571,439","31,167,662,774","-1,605,158,795",0,0,"9,036,463,600",0,"-2,198,636,640",0


```javascrip
function ClickConnect(){
    console.log("코랩 연결 끊김 방지"); 
    document.querySelector("colab-toolbar-button#connect").click() 
}
setInterval(ClickConnect, 60 * 1000)
```

In [148]:
포괄손익누락추정 = pd.DataFrame(columns=['종목코드','사업연도', '보고서종류', '회사명', '업종', '항목코드','항목명', '당기'])
for path in tqdm(포괄손익계산서):
  df = pd.read_csv(path, delimiter = '\t', encoding='cp949')

  당기 = ''
  if df['보고서종류'][0] == '1분기보고서':
    당기 = '당기 1분기 누적' 
  elif df['보고서종류'][0] == '반기보고서':
    당기 = '당기 반기 누적' 
  elif df['보고서종류'][0] == '3분기보고서':
    당기 = '당기 3분기 누적' 
  elif df['보고서종류'][0] == '사업보고서':
    당기 = '당기' 

  df['사업연도'] = path[57:61] # 사업연도 컬럼 설정 - 파일명에서 가져온다
  df['항목명'] = df['항목명'].str.strip() # 항목명 컬럼 문자열 전처리
  df['항목코드'] = df['항목코드'].str.strip() # 항목코드 컬럼 문자열 전처리

  df = df[['종목코드','사업연도', '보고서종류', '회사명', '업종', '항목코드','항목명', 당기]]

  전체종목 = len(df['종목코드'].unique())

  # (수익)매출액 (ifrs-full_Revenue)
  포괄손익누락추정 = 포괄손익계산샘플.append(df, sort=False, ignore_index=True)

100%|██████████| 2/2 [00:01<00:00,  1.72it/s]


In [149]:
포괄손익누락추정['종목코드'] = 포괄손익누락추정.apply(lambda x: x['종목코드'][1:-1] if '[' in x['종목코드'] else x['종목코드'], axis=1)
포괄손익누락추정

,종목코드,사업연도,보고서종류,회사명,업종,항목코드,항목명,당기,당기 1분기 누적
0,060310,2021,사업보고서,3S,292,ifrs_Revenue,매출액,"23,477,908,383",NaN
1,095570,2021,사업보고서,AJ네트웍스,763,ifrs_Revenue,매출액,"1,026,749,457,003",NaN
2,006840,2021,사업보고서,AK홀딩스,649,ifrs_Revenue,매출액,"3,167,699,652,090",NaN
3,054620,2021,사업보고서,APS홀딩스,649,ifrs_Revenue,매출액,"39,415,521,257",NaN
4,265520,2021,사업보고서,AP시스템,292,ifrs_Revenue,매출액,"528,740,970,343",NaN
...,...,...,...,...,...,...,...,...,...
68681,238490,2022,1분기보고서,힘스,292,ifrs-full_ComprehensiveIncomeAttributableToAbs...,총 포괄손익의 귀속 [abstract],NaN,NaN
68682,238490,2022,1분기보고서,힘스,292,ifrs-full_ComprehensiveIncomeAttributableToOwn...,"총 포괄손익, 지배기업의 소유주에게 귀속되는 지분",NaN,"-1,604,426,026"
68683,238490,2022,1분기보고서,힘스,292,ifrs-full_EarningsPerShareAbstract,주당이익 [abstract],NaN,NaN
68684,238490,2022,1분기보고서,힘스,292,ifrs-full_BasicEarningsLossPerShare,기본주당이익(손실),NaN,-144


Error: Runtime no longer has a reference to this dataframe, please re-run this cell and try again.


In [150]:
len(포괄손익누락추정['종목코드'].unique().tolist())

1850

In [153]:
포괄손익누락추정[포괄손익누락추정['종목코드'].isin(매출액누락)]

,종목코드,사업연도,보고서종류,회사명,업종,항목코드,항목명,당기,당기 1분기 누적
228,950220,2021,사업보고서,네오이뮨텍,701,ifrs_Revenue,매출액,0,NaN
449,127160,2021,사업보고서,매직마이크로,262,ifrs_Revenue,매출액,0,NaN
454,235980,2021,사업보고서,메드팩토,701,ifrs_Revenue,매출액,0,NaN
552,357880,2021,사업보고서,비트나인,582,ifrs_Revenue,매출액,"16,414,865,507",NaN
659,052190,2021,사업보고서,세영디앤씨,901,ifrs_Revenue,매출액,"386,726,283",NaN
...,...,...,...,...,...,...,...,...,...
68681,238490,2022,1분기보고서,힘스,292,ifrs-full_ComprehensiveIncomeAttributableToAbs...,총 포괄손익의 귀속 [abstract],NaN,NaN
68682,238490,2022,1분기보고서,힘스,292,ifrs-full_ComprehensiveIncomeAttributableToOwn...,"총 포괄손익, 지배기업의 소유주에게 귀속되는 지분",NaN,"-1,604,426,026"
68683,238490,2022,1분기보고서,힘스,292,ifrs-full_EarningsPerShareAbstract,주당이익 [abstract],NaN,NaN
68684,238490,2022,1분기보고서,힘스,292,ifrs-full_BasicEarningsLossPerShare,기본주당이익(손실),NaN,-144
